# Table 9: Macro-average Accuracy with different retraining dataset and different sampling methods

[:simple-jupyter: :material-download:](../../paper_tables_and_figures/table9_icdm_finetuning_per_class_metrics_on_human/table9_icdm_finetuning_per_class_metrics_on_human.ipynb)

In [1]:
import pathlib

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.stats.api as sms

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
def compute_confidence_intervals(array, alpha=0.05):
    array = np.array(array)
    low, high = sms.DescrStatsW(array).tconfint_mean(alpha)
    mean = array.mean()
    ci = high - mean
    return ci

In [3]:
path = pathlib.Path("./campaigns/ucdavis-icdm19-git-repo-forked/artifacts/")

class_repss = list(path.glob("*10/"))

In [4]:
data = dict()

for path in class_repss:
    if "script" in str(path):
        class_reps = list(path.glob("*class_rep.csv"))
        accs = [pd.read_csv(file).iloc[6].values[2] for file in class_reps]

        augmentation_name = path.name.split("_")[0].replace("Sampling", "")
        data[augmentation_name] = (
            np.mean(accs) * 100,
            compute_confidence_intervals(accs),
        )

df_script = pd.DataFrame(data, index=["mean", "ci95"]).T.round(2)
df_script.columns = pd.MultiIndex.from_arrays([["script", "script"], df_script.columns])
# df_script

In [5]:
data = dict()
for path in class_repss:
    if "human" in str(path):
        class_reps = list(path.glob("*class_rep.csv"))
        accs = [pd.read_csv(file).iloc[6].values[2] for file in class_reps]

        augmentation_name = path.name.split("_")[0].replace("Sampling", "")
        data[augmentation_name] = (
            np.mean(accs) * 100,
            compute_confidence_intervals(accs),
        )

df_human = pd.DataFrame(data, index=["mean", "ci95"]).T.round(2)
df_human.columns = pd.MultiIndex.from_arrays([["human", "human"], df_human.columns])

In [7]:
df_tmp = pd.concat((df_script, df_human), axis=1).T
display(df_tmp)
df_tmp.to_csv("icdm_finetuning_per_class_metrics_on_human.csv")

FixedStep  Random  Incremental
script mean      87.11   94.63        96.22
       ci95       0.09    0.02         0.01
human  mean      82.60   87.29        92.56
       ci95       0.03    0.04         0.03